# How to complete this assignment
First, download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. You may need to register at https://www.kaggle.com/ beforehand. Then complete 15 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.

- Do not delete or rename the variables given before the inscription `#your code here`, they are needed for the correct verification.
- Do not change the code in the last Notebook cell, it is required for the server check.
- Your Notebook must run completely without errors to be graded! Please check everything before submission by going *Cell -> Run All*

## Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [1]:
import sqlite3
import pandas as pd
import os

pd.set_option('display.max_column', None)

In [2]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [3]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)

In [4]:
league_data = pd.read_sql("SELECT * FROM League;", db)
country_data = pd.read_sql("SELECT * FROM Country;", db)
match_data = pd.read_sql("SELECT * FROM Match;", db)
team_data = pd.read_sql("SELECT * FROM Team;", db)

In [5]:
attributes_data = pd.read_sql("SELECT * FROM Player_Attributes;", db)

In [6]:
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


In [7]:
player_data.shape

(11060, 7)

**Task 1 (0.25 point).** Calculate the number of players with a height between 180 and 190 inclusive

In [8]:
players_180_190 = int(player_data.query("180 <= height <= 190").id.count())

In [9]:
assert(isinstance(players_180_190, int))

**Task 2 (0.25 point).** Calculate the number of players born in 1980. <br>
**Hint:** you may want to cast your 'birthday' column to DateTime type by [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [10]:
player_data.birthday = pd.to_datetime(player_data.birthday)

In [11]:
player_data['year'] = pd.DatetimeIndex(player_data['birthday']).year

In [12]:
player_data[player_data.year == 1980].id.count()

434

In [13]:
players_1980 = int(player_data[player_data.year == 1980].id.count())

In [14]:
assert(isinstance(players_1980, int))

**Task 3 (0.25 point).** Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.

In [15]:
highest_players = player_data.sort_values(["weight", "player_name"], \
    ascending=[False, True]) \
    ["player_name"].head(10).values.tolist()

In [16]:
assert(len(highest_players) == 10)
assert(isinstance(highest_players, list))
for i in range(10):
    assert(isinstance(highest_players[i], str))

**Task 4 (0.5 point).** Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990. <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [17]:
years = player_data.query("1980 <= year <= 1990").groupby("year")["id"].count()

In [18]:
years_born_players = list(zip(years.index, years.values.tolist()))

In [19]:
assert(len(years_born_players) == 11)
assert(isinstance(years_born_players, list))
for i in range(10):
    assert(isinstance(years_born_players[i], tuple))
    assert(isinstance(years_born_players[i][0], int))
    assert(isinstance(years_born_players[i][1], int))

**Task 5 (0.5 point).** Calculate the mean and the standard deviation of the players' **height** with the name **Adriano**. <br>
**Note:** Name is represented by the first part of `player_name`.

In [20]:
def get_name(row):
    row.player_name = row.player_name.split()[0]
    return row

In [21]:
names = player_data.apply(get_name, axis='columns')

In [22]:
names[names["player_name"] == "Adriano"].height.mean()

182.11800000000002

In [23]:
adriano_mean, adriano_std = names[names["player_name"] == "Adriano"].height.mean(), \
    names[names["player_name"] == "Adriano"].height.std()

In [24]:
assert(isinstance(adriano_mean, float))
assert(isinstance(adriano_std, float))

**Task 6 (0.75 point).** How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [25]:
player_data['day_of_the_week'] = pd.DatetimeIndex(player_data['birthday']).dayofweek

In [26]:
days = list(zip(player_data.groupby("day_of_the_week").id.count().index, \
         player_data.groupby("day_of_the_week").id.count().values.tolist()))

In [27]:
keys = {
        0 : "Monday", 1 : "Tuesday", 
        2 : "Wednesday", 3 : "Thursday", 4 : "Friday",
        5 : "Saturday", 6 : "Sunday"        
       }

In [28]:
dow_with_min_players_born = keys[sorted(days, key = lambda x: x[1])[0][0]]

In [29]:
assert(isinstance(dow_with_min_players_born, str))

**Task 7 (0.75 point).** Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [30]:
result = match_data.groupby("league_id").id.count()

In [31]:
max_matches = match_data.groupby("league_id").id.count().max()

In [32]:
max_match_leagues_id = result[result.values == max_matches].index.values.tolist()

In [33]:
max_match_leagues_id

[1729, 4769, 21518]

In [34]:
leagues = league_data[league_data.id.isin(max_match_leagues_id)].name.values.tolist()

In [35]:
res = sorted(leagues)[0]

In [36]:
league_most_matches = res

In [37]:
assert(isinstance(league_most_matches, str))

**Task 8 (1.25 point).** Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [38]:
cols1 = [f"home_player_{i}" for i in range(1, 12)]
cols2 = [f"away_player_{i}" for i in range(1, 12)]
#cols = ["id"] + cols1+cols2
cols = cols1+cols2

In [39]:
#res = match_data[cols].to_numpy().flatten()

In [40]:
#from collections import Counter
#res = Counter(res).most_common(1)

In [41]:
player_data[player_data["player_api_id"] == 31293]

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight,year,day_of_the_week
9993,10009,31293,Steve Mandanda,163705,1985-03-28,185.42,181,1985,3


In [42]:
max_matches_player = "Steve Mandanda"

In [43]:
assert(isinstance(max_matches_player, str))

**Task 9 (1.5 point).** List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [75]:
cols_to_drop = ["id", "player_fifa_api_id", "player_api_id", "date", "preferred_foot", "attacking_work_rate", "defensive_work_rate"]

In [76]:
attributes_data_new = attributes_data.drop(cols_to_drop, axis=1)

In [77]:
corr = attributes_data_new.corr().abs()

In [78]:
s = corr.unstack()

In [79]:
so = s.sort_values(kind="quicksort", ascending = False)

In [80]:
a = so[so.values != 1]

In [81]:
a.index[:10:2]

MultiIndex([(    'gk_reflexes',  'gk_positioning'),
            (    'gk_reflexes',     'gk_handling'),
            (    'gk_handling',  'gk_positioning'),
            ( 'sliding_tackle', 'standing_tackle'),
            ('standing_tackle',         'marking')],
           )

In [50]:
top_correlated_features = [('gk_reflexes', 'gk_positioning'), ('gk_reflexes', 'gk_handling'),
                          ('gk_handling', 'gk_positioning'), ('sliding_tackle', 'standing_tackle'),
                          ('standing_tackle', 'marking')]

In [51]:
assert(len(top_correlated_features) == 5)
assert(isinstance(top_correlated_features, list))
for i in range(5):
    assert(isinstance(top_correlated_features[i], tuple))
    assert(isinstance(top_correlated_features[i][0], str))
    assert(isinstance(top_correlated_features[i][1], str))

**Task 10 (2 points).** Find top-5 most similar players to **Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same<br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [161]:
neymar_id = player_data[player_data["player_name"] == "Neymar"].player_api_id.values.tolist()

In [162]:
all_players = attributes_data.player_api_id.unique().tolist()

In [163]:
neymar_attrs = attributes_data[attributes_data["player_api_id"].isin(neymar_id)].drop(cols_to_drop, axis=1).iloc[0,:].values

In [164]:
#results = [(i, 
#            np.linalg.norm(neymar_attrs - attributes_data[attributes_data["player_api_id"] == i].drop(cols_to_drop, axis=1).fillna(value=0).iloc[0,:].values)) \
#          for i in all_players]

In [172]:
#similar_players = sorted(results, key=lambda x: x[1])[1:6]

In [176]:
#sorted(results, key=lambda x: x[1])[1:6]

[(325916, 33.83784863137726),
 (107417, 34.95711658589707),
 (41157, 38.31448812133603),
 (30834, 40.024992192379),
 (30981, 43.54308211415448)]

In [173]:
#similar_players = [i[0] for i in similar_players]

In [174]:
#player_data[player_data["player_api_id"].isin(similar_players)]

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight,year,day_of_the_week
948,951,30834,Arjen Robben,9014,1984-01-23,180.34,176,1984,0
2835,2838,107417,Eden Hazard,183277,1991-01-07,172.72,163,1991,0
3869,3873,41157,Giovani dos Santos,170369,1989-05-11,175.26,163,1989,3
6169,6176,30981,Lionel Messi,158023,1987-06-24,170.18,159,1987,2
8423,8436,325916,Paulo Dybala,211110,1993-11-15,175.26,161,1993,0


In [134]:
neymar_similarities = ["Paulo Dybala",'Eden Hazard',"Giovani dos Santos","Arjen Robben","Lionel Messi"]

In [136]:
assert(len(neymar_similarities) == 5)
assert(isinstance(neymar_similarities, list))
for i in range(5):
    assert(isinstance(neymar_similarities[i], str))

**Task 11 (1 point).** Calculate the number of home matches played by the **Borussia Dortmund** team in **Germany 1. Bundesliga** in season **2008/2009**

In [182]:
borrusia_id = int(team_data[team_data["team_long_name"] == "Borussia Dortmund"].team_api_id)

In [193]:
match_data[(match_data["home_team_api_id"] == borrusia_id) & (match_data["season"] == "2008/2009")].id.count()

17

In [194]:
borussia_bundesliga_2008_2009_matches = 17

In [195]:
assert(isinstance(borussia_bundesliga_2008_2009_matches, int))

**Task 12 (1 point).** Find a team having the most matches (both home and away!) in the **Germany 1. Bundesliga** in **2008/2009** season. Return number of matches.

In [198]:
bundes2008_teams = match_data[(match_data["league_id"] == 7809) & (match_data["season"] == "2008/2009")][["home_team_api_id", "away_team_api_id"]]

In [212]:
res = bundes2008_teams.to_numpy().flatten()

In [215]:
from collections import Counter
res = Counter(res).most_common(1)

In [216]:
res[0][1]

34

In [217]:
team_most_matches_bundesliga_2008_2009 = 34

In [218]:
assert(isinstance(team_most_matches_bundesliga_2008_2009, int))

**Task 13 (1 point).** Count total number of **Arsenal** matches (both home and away!) in the **2015/2016** season which they have won. <br><br>
**Note:** Winning a game means scoring **more** goals than an opponent.

In [227]:
arsenal_id = int(team_data[team_data["team_long_name"] == "Arsenal"].team_api_id)

In [221]:
season_2015 = match_data.iloc[:, :11]

In [223]:
season_2015["home_win"] = season_2015["home_team_goal"] > season_2015["away_team_goal"]
season_2015["away_win"] = season_2015["away_team_goal"] > season_2015["home_team_goal"]

In [225]:
season_2015 = season_2015[season_2015["season"] == "2015/2016"]

In [233]:
query = f"(home_team_api_id == {arsenal_id} and home_win) or (away_team_api_id == {arsenal_id} and away_win)"

In [240]:
ars_win = int(season_2015.query(query).id.count())

In [241]:
arsenal_won_matches_2015_2016 = ars_win

In [242]:
assert(isinstance(arsenal_won_matches_2015_2016, int))

**Task 14 (2 points).** Find a team with the highest win rate in the **2015/2016** season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [266]:
all_teams = set(season_2015[["away_team_api_id", "home_team_api_id"]].to_numpy().flatten())

In [ ]:
winrate = []
for team in all_teams:
    query_win = f"(home_team_api_id == {team} and home_win) or (away_team_api_id == {team} and away_win)"
    query_all = f"home_team_api_id == {team} or away_team_api_id == {team}"
    win = season_2015.query(query_win).id.count()
    alls = season_2015.query(query_all).id.count()
    rate = win / alls
    winrate.append((team, rate))

In [284]:
most_wins_team_id = sorted(winrate, key = lambda x: x[1], reverse = True)[0][0]

In [294]:
team = str(team_data[team_data["team_api_id"] == most_wins_team_id].team_long_name.values.tolist()[0])

In [296]:
team_highest_winrate_2015_2016 = team

In [292]:
assert(isinstance(team_highest_winrate_2015_2016, str))

**Task 15 (2 points).** Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season**. Return number of days in that gap. <br>
**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [382]:
from datetime import datetime

In [383]:
epl_2010 = match_data[(match_data["league_id"] == 1729) & (match_data["season"] == "2010/2011")]

In [384]:
all_teams = set(epl_2010[["away_team_api_id", "home_team_api_id"]].to_numpy().flatten())

In [385]:
epl_2010 = epl_2010.iloc[:, :9]

In [398]:
max_gap = 0
for team in all_teams:    
    one = epl_2010[(epl_2010["home_team_api_id"] == team) \
         | (epl_2010["away_team_api_id"] == team)] \
        .sort_values(by = "date").date.values.tolist()
    
    dates = [datetime.strptime(i.split()[0], "%Y-%m-%d") for i in one]
    gap = max([abs(dates[i] - dates[i - 1]).days for i in range(1, len(dates))])    
    if gap > max_gap:
        max_gap = gap
   

In [399]:
highest_gap_england_2010_2011 = max_gap

In [400]:
assert(isinstance(highest_gap_england_2010_2011, int))

### Warning! Do not change anything in the area below

In [ ]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")